## PASO 1: Plantear la pregunta.
        ¿Puede predecirse el consumo energético de un país en función de su población y su nivel de industrialización?
## PASOS 2 y 3: Concretar y buscar los datos necesarios.
        Cargamos los ficheros .csv utilizando las librerías y funciones correspondientes.           

In [1]:
import pandas as pd

#Ficheros de datos
pop_den = pd.read_csv("population_density.csv")
urb_gro = pd.read_csv("urban_growth.csv")
pop = pd.read_csv("population.csv")
pop_gro = pd.read_csv("population_growth.csv")
life_exp = pd.read_csv("life_expectancy.csv")
co2_emi = pd.read_csv("co2_emissions.csv")

#Fichero de comprobación
target = pd.read_csv("energy_person_ratio.csv")

## PASOS 4 y 5: Limpieza y exploración de los datos..
        1) Transponemos las tablas utilizando la función data.melt_df().
        2) Juntamos las tablas mediante la función data.merge_all(), describiendo nuestro propio diccionario.
        3) Visualizamos la información básica de nuestra tabla global mediante las funciones .head() y .describe() 

In [2]:
import data #data es una librería de funciones propias que nos permiten unir y transponer los dataframes anteriores de manera fácil y rápida (Si quieres profundizar, abre el fichero data.py) 
pop_den_melt = data.melt_df(pop_den, "population_density")
urb_gro_melt = data.melt_df(urb_gro, "urban_growth")
pop_melt = data.melt_df(pop, "population")
pop_gro_melt = data.melt_df(pop_gro, "population_growth")
co2_emi_melt = data.melt_df(co2_emi, "co2_emissions")
life_exp_melt = data.melt_df(life_exp, "life_expectancy")

target_melt = data.melt_df(target, "target")

In [3]:
#Describimos un objeto tipo diccionario feature_dict, que usaremos como argumento en la función data.merge_all()
feature_dict = {"pop_den_melt":pop_den_melt, "urb_gro_melt": urb_gro_melt, "pop_melt": pop_melt, "pop_gro_melt": pop_gro_melt, "co2_emi_melt": co2_emi_melt, "life_exp_melt": life_exp_melt}

merged_data = data.merge_all(feature_dict = feature_dict, keys = ["country", "year"])

In [4]:
merged_data.head()

,country,year,population_density,urban_growth,population,population_growth,co2_emissions,life_expectancy
2145,Afghanistan,1961,14.00,0.052100,9170000,1.90,0.0536,40.0
2146,Albania,1961,61.50,0.038900,1690000,3.12,1.3500,63.3
2147,Algeria,1961,4.76,0.066200,11300000,2.49,0.5350,53.1
2149,Angola,1961,4.44,0.048100,5530000,1.39,0.0821,41.1
2150,Antigua and Barbuda,1961,125.00,0.000279,55000,1.59,0.8670,63.8


In [5]:
merged_data.describe()

,population_density,urban_growth,population,population_growth,co2_emissions,life_expectancy
count,10501.000000,10501.000000,1.050100e+04,10501.000000,10501.000000,10501.000000
mean,132.850914,0.030194,2.897746e+07,1.807020,4.482479,65.505327
std,406.795876,0.030978,1.120533e+08,1.576097,7.341779,10.128384
min,0.632000,-1.870000,4.220000e+04,-9.080000,0.000000,9.640000
25%,17.700000,0.013000,1.820000e+06,0.748000,0.385000,58.300000
50%,55.200000,0.027700,5.730000e+06,1.770000,1.730000,68.100000
75%,122.000000,0.043500,1.750000e+07,2.700000,6.270000,73.200000
max,8230.000000,0.489000,1.430000e+09,17.600000,101.000000,85.000000
